In [1]:
! pip install spacy scikit-learn
! python -m spacy download en_core_web_sm


2023-11-27 17:06:46.851349: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 17:06:46.851433: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 17:06:46.851484: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 17:06:46.865798: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 17:06:48.801620: W tensorflow/compiler/

In [3]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocessing function using spaCy
def preprocess(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

def classify_news(subject, df, method='randomforest'):
    subject_df = df[df['subject'] == subject]

    #subject_df['processed_summary'] = subject_df['summary'].apply(preprocess)
    subject_df['processed_title'] = subject_df['title'].apply(preprocess)
    #subject_df['processed_description'] = subject_df['description'].apply(preprocess)

    # Combine 'subject' and 'processed_summary' as features
    subject_df['features'] = subject_df['processed_title']

    # Feature extraction using TF-IDF
    tfidf = TfidfVectorizer(max_features=1000)
    X = tfidf.fit_transform(subject_df['features'])
    y = subject_df['action']

    # Splitting the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Selecting the model based on the method argument
    if method == 'randomforest':
        model = RandomForestClassifier()
    elif method == 'logistic':
        model = LogisticRegression()
    elif method == 'svm':
        model = SVC()
    elif method == 'knn':
        model = KNeighborsClassifier()
    elif method == 'decisiontree':
        model = DecisionTreeClassifier()
    else:
        raise ValueError("Unknown method provided.")
    model.fit(X_train, y_train)

    # Model evaluation
    y_pred = model.predict(X_test)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    return report['accuracy'], report['macro avg']['support']


In [6]:
import pandas as pd

# Assuming classify_news is defined as in the previous script
# ...

df = pd.read_csv('news_price.csv')
# Drop rows where 'summary', 'subject', or 'action' are NaN
df = df.dropna(subset=['summary', 'subject', 'action'])

subject_counts = df['subject'].value_counts().to_dict()

#methods = ['randomforest', 'logistic', 'svm', 'knn', 'decisiontree']
methods = ['randomforest']
results = []

for method in methods:
    for subject in df['subject'].unique():
        print(f"Processing subject: {subject} with method: {method}")
        try:
            accuracy, support = classify_news(subject, df, method=method)
            results.append({
                'subject': subject,
                'accuracy': accuracy,
                'test_sample': support,
                'total_sample': subject_counts[subject],  # Total count of the subject
                'method': method
            })
        except Exception as e:
            print(f"Error processing subject {subject} with method {method}: {e}")

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by=['accuracy', 'method'], ascending=[False, True])
results_df.to_csv('classify_news_results.csv')
print(results_df.head())


Processing subject: Calendar of Events with method: randomforest


<ipython-input-3-47af0dd23cab>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['processed_title'] = subject_df['title'].apply(preprocess)
<ipython-input-3-47af0dd23cab>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['features'] = subject_df['processed_title']


Processing subject: Health with method: randomforest


<ipython-input-3-47af0dd23cab>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['processed_title'] = subject_df['title'].apply(preprocess)
<ipython-input-3-47af0dd23cab>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['features'] = subject_df['processed_title']
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` para

Processing subject: Company Announcement with method: randomforest


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

Processing subject: Pre-Release Comments with method: randomforest


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

Processing subject: Changes in share capital and votes with method: randomforest
Processing subject: Feature Article with method: randomforest
Error processing subject Feature Article with method randomforest: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
Processing subject: Exchange announcement with method: randomforest
Error processing subject Exchange announcement with method randomforest: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
Processing subject: Company Regulatory Filings with method: randomforest
Error processing subject Company Regulatory Filings with method randomforest: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
Processing subject: Restructuring / Recapitalization with method: randomforest
Error pr

<ipython-input-3-47af0dd23cab>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['features'] = subject_df['processed_title']
<ipython-input-3-47af0dd23cab>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['processed_title'] = subject_df['title'].apply(preprocess)
<ipython-input-3-47af0dd23cab>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [8]:
# Assuming 'results_df' is the DataFrame with the classification results and 'df' is the original 'news_price' DataFrame
filtered_df = results_df[results_df['accuracy'] > 0.6]
# Get the list of subjects with accuracy greater than 60%
subjects_above_60 = filtered_df['subject'].unique()

# Use this list to filter the original 'news_price' DataFrame
filtered_news_price_df = df[df['subject'].isin(subjects_above_60)]
filtered_news_price_df.head()  # Display the first few rows of the filtered DataFrame



,title,summary,description,subject,return,daily_alpha,action
2,AC Immune Awarded New Grants from MJFF and Tar...,AC Immune Awarded New Grants from MJFF and Tar...,AC Immune Awarded New Grants from MJFF and Tar...,Health,0.012658,0.015001,long
6,AC Immune Reports First Quarter 2023 Financial...,AC Immune Reports First Quarter 2023 Financial...,AC Immune Reports First Quarter 2023 Financial...,Interim information,0.000000,0.002231,long
25,Adagene Achieves $3 Million Milestone in Colla...,"SAN DIEGO and SUZHOU, China, May 04, 2023 (G...","SAN DIEGO and SUZHOU, China, May 04, 2023 (G...",Product / Services Announcement,0.000000,0.002671,long
31,Adagene Presents Data Demonstrating the Best-i...,- MSS CRC case examples reinforce optimal dosi...,- MSS CRC case examples reinforce optimal dosi...,Product / Services Announcement,0.015038,0.011459,long
59,AEON Biopharma Presents Positive Results from ...,"IRVINE, Calif., Aug. 29, 2023 (GLOBE NEWSWIR...","IRVINE, Calif., Aug. 29, 2023 (GLOBE NEWSWIR...",Research Analysis and Reports,0.038929,0.039178,long


In [12]:
# Assuming filtered_news_price_df['daily_alpha'] contains the original daily alpha values

# Create a new column for percentage daily_alpha values
filtered_news_price_df['percent_daily_alpha'] = 100 * filtered_news_price_df['daily_alpha']

# Create a frequency distribution bar chart for the percentage daily_alpha values
fig = px.histogram(filtered_news_price_df, x='percent_daily_alpha', nbins=50, title='Frequency Distribution of Daily Alpha (%)')

# Update the layout for clarity, including more detailed tick labels
fig.update_layout(
    xaxis_title='Daily Alpha (%)',
    yaxis_title='Count',
    bargap=0.2,
    xaxis=dict(
        tickmode='linear',
        tick0=0,
        dtick=5  # Set the interval between ticks to 5%
    )
)

# Show the plot
fig.show()



<ipython-input-12-8accf30d6368>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
import plotly.express as px

alpha_stdev_df = filtered_news_price_df.groupby('subject')['daily_alpha'].std().reset_index()
alpha_stdev_df.rename(columns={'daily_alpha': 'alpha_std'}, inplace=True)


fig = px.bar(alpha_stdev_df, x='subject', y='alpha_std',
             title='Standard Deviation of Daily Alpha by Subject Category',
             labels={'alpha_std': 'Standard Deviation of Daily Alpha', 'subject': 'Subject Category'},
             color='alpha_std',
             color_continuous_scale=px.colors.sequential.Viridis)

fig.update_layout(xaxis={'categoryorder': 'total descending'}, xaxis_title='Subject Category', yaxis_title='Standard Deviation of Daily Alpha')
fig.show()
